## Code to Generate TIF from Google Earth Engine
### Requirements: 
- Google Earth Engine Project
- Google Drive access (to store the accuracy metrics in a csv)
- Google drive access (to store the generated TIFs)
- A `coordinates.xlsx` file - to copy the coordinates from 

### Initialize

In [1]:
import ee
import csv
import os

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='omkarsoak-ee')

In [2]:
from google.colab import drive
drive.mount('/content/drive')
csv_filename = '/content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv'

### COMBINED RGB + MASK SVM

In [3]:
def process_point_svm(lon, lat, city, state, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-07-01', f'{year}-09-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
                 .median())

    # Calculate indices and masks
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
    swm = sentinel2.expression(
        '(B2 + B3) / (B8 + B11)',
        {
            'B2': sentinel2.select('B2'),
            'B3': sentinel2.select('B3'),
            'B8': sentinel2.select('B8'),
            'B11': sentinel2.select('B11')
        }
    ).rename('SWM')

    # Combine NDVI and NDWI into a single image for sampling
    feature_image = ndvi.addBands(swm)

    # Define class masks
    water_mask = swm.gt(1.5)
    building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
    veg_mask = ndvi.gte(0.4)
    # dense_veg_mask = ndvi.gte(0.7)

    # Sample points for each class
    NUM_OF_PIXELS = 70000

    def sample_points(mask, class_value):
        return feature_image.updateMask(mask).sample(
            region=geometry,
            scale=10,
            numPixels=NUM_OF_PIXELS,
            geometries=True
        ).map(lambda f: f.set('class', class_value))

    water_points = sample_points(water_mask, 0)
    building_points = sample_points(building_mask, 1)
    veg_points = sample_points(veg_mask, 2)
    # dense_veg_points = sample_points(dense_veg_mask, 3)

    # Merge all training points
    training_points = water_points.merge(building_points).merge(veg_points)

    # Split the data into training (80%) and validation (20%) sets
    with_random = training_points.randomColumn()
    split = 0.8
    training_data = with_random.filter(ee.Filter.lt('random', split))
    validation_data = with_random.filter(ee.Filter.gte('random', split))

    # Train an SVM classifier
    svm_classifier = ee.Classifier.libsvm().train(
        features=training_data,
        classProperty='class',
        inputProperties=['NDVI', 'SWM']
    )

    # Classify the image
    classified_image = feature_image.classify(svm_classifier)
    # classified_image = classified_image.toUint16()

    # # Evaluate the classifier using the validation data
    # validation = validation_data.classify(svm_classifier)
    # validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    # print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
    # print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

    # # Compute training accuracy
    # training_confusion_matrix = svm_classifier.confusionMatrix()
    # print('Training Error Matrix:', training_confusion_matrix.getInfo())
    # print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())


    # Evaluate the classifier using the validation data
    validation = validation_data.classify(svm_classifier)
    validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    validation_accuracy = validation_confusion_matrix.accuracy().getInfo()

    # Compute training accuracy
    training_confusion_matrix = svm_classifier.confusionMatrix()
    training_accuracy = training_confusion_matrix.accuracy().getInfo()

    # # Save results to CSV
    # csv_filename = "classification_results.csv"
    header = ["City", "State", "Year", "Filename",
              "Training Accuracy", "Training Confusion Matrix",
              "Validation Accuracy", "Validation Confusion Matrix"]

    data = [
        city, state, year, filename,
        training_accuracy, training_confusion_matrix.getInfo(),
        validation_accuracy, validation_confusion_matrix.getInfo()
    ]

    # Write to CSV (append if file exists, otherwise create)
    try:
        with open(csv_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:  # File is empty, write header
                writer.writerow(header)
            writer.writerow(data)
    except Exception as e:
        print(f"Error writing to CSV: {e}")

    # print(f"Results saved to {csv_filename}.")

    # try:
    #   file_exists = os.path.isfile(csv_filename)
    #   with open(csv_filename, 'a', newline='') as file:
    #       writer = csv.writer(file)
    #       if not file_exists:  # If the file doesn't exist, write the header
    #           writer.writerow(header)
    #       writer.writerow(data)
    # except Exception as e:
    #   print(f"Error writing to CSV: {e}")

    print(f"Results saved to {csv_filename}.")

    # Enhanced RGB export
    enhanced_rgb = sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0)
    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export tasks
    # 1. Enhanced RGB
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.uint16().clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}_{year}',
        folder='ChangeDetectionEur',
        fileNamePrefix=f'{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # 2. Classified Image
    class_vis = {
        'min': 0,
        'max': 2,
        'palette': ['lightblue', 'white', 'lightgreen']
    }
    mask_task = ee.batch.Export.image.toDrive(
        image=classified_image.uint16(),
        description=f'm_{filename}_{year}',
        folder='ChangeDetectionEur',
        fileNamePrefix=f'm_{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # Start both export tasks
    enhanced_rgb_task.start()
    mask_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")

### Import data from csv
- directly copy paste into a cell
- just do `data = pd.read_csv(...`

In [11]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
Netherlands,Rotterdam,2019,2024,"4.47777963162828,51.9222407862643",51.92224079,4.477779632,"4.47777963162828,51.9722407862643","4.52777963162828,51.9722407862643","4.52777963162828,51.9222407862643","4.52777963162828,51.8722407862643","4.47777963162828,51.8722407862643","4.42777963162828,51.8722407862643","4.42777963162828,51.9222407862643","4.42777963162828,51.9722407862643"
Norway,Oslo,2019,2024,"10.7954092160319,59.9271445180858",59.92714452,10.79540922,"10.7954092160319,59.9771445180858","10.8454092160319,59.9771445180858","10.8454092160319,59.9271445180858","10.8454092160319,59.8771445180858","10.7954092160319,59.8771445180858","10.7454092160319,59.8771445180858","10.7454092160319,59.9271445180858","10.7454092160319,59.9771445180858"
Poland,Wroclaw,2019,2024,"17.0313825224446,51.110014588585",51.11001459,17.03138252,"17.0313825224446,51.160014588585","17.0813825224446,51.160014588585","17.0813825224446,51.110014588585","17.0813825224446,51.060014588585","17.0313825224446,51.060014588585","16.9813825224446,51.060014588585","16.9813825224446,51.110014588585","16.9813825224446,51.160014588585"
Poland,Krakow,2019,2024,"19.9567179575101,50.0588340571982",50.05883406,19.95671796,"19.9567179575101,50.1088340571982","20.0067179575101,50.1088340571982","20.0067179575101,50.0588340571982","20.0067179575101,50.0088340571982","19.9567179575101,50.0088340571982","19.9067179575101,50.0088340571982","19.9067179575101,50.0588340571982","19.9067179575101,50.1088340571982"
Poland,Lodz,2019,2024,"19.4643562223554,51.7648092131436",51.76480921,19.46435622,"19.4643562223554,51.8148092131436","19.5143562223554,51.8148092131436","19.5143562223554,51.7648092131436","19.5143562223554,51.7148092131436","19.4643562223554,51.7148092131436","19.4143562223554,51.7148092131436","19.4143562223554,51.7648092131436","19.4143562223554,51.8148092131436"
Poland,Poznan,2019,2024,"16.914742312342,52.4049821958422",52.4049822,16.91474231,"16.914742312342,52.4549821958422","16.964742312342,52.4549821958422","16.964742312342,52.4049821958422","16.964742312342,52.3549821958422","16.914742312342,52.3549821958422","16.864742312342,52.3549821958422","16.864742312342,52.4049821958422","16.864742312342,52.4549821958422"
Poland,Warsaw,2019,2024,"21.0112859757535,52.232466557807",52.23246656,21.01128598,"21.0112859757535,52.282466557807","21.0612859757535,52.282466557807","21.0612859757535,52.232466557807","21.0612859757535,52.182466557807","21.0112859757535,52.182466557807","20.9612859757535,52.182466557807","20.9612859757535,52.232466557807","20.9612859757535,52.282466557807"
Portugal,Lisbon,2019,2024,"-9.17003829572333,38.7550284336539",38.75502843,-9.170038296,"-9.17003829572333,38.8050284336539","-9.12003829572333,38.8050284336539","-9.12003829572333,38.7550284336539","-9.12003829572333,38.7050284336539","-9.17003829572333,38.7050284336539","-9.22003829572333,38.7050284336539","-9.22003829572333,38.7550284336539","-9.22003829572333,38.8050284336539"
Romania,Bucharest,2019,2024,"26.0903292835383,44.4279852258696",44.42798523,26.09032928,"26.0903292835383,44.4779852258696","26.1403292835383,44.4779852258696","26.1403292835383,44.4279852258696","26.1403292835383,44.3779852258696","26.0903292835383,44.3779852258696","26.0403292835383,44.3779852258696","26.0403292835383,44.4279852258696","26.0403292835383,44.4779852258696"
'''), header=None)

### Run

In [12]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]

# Process each city
for j in range(ncities):
    cityData = data.iloc[j]
    city = cityData[1]
    state = cityData[0]
    print(city, state)

    for i in range(9):
        try:
            coord = cityData.iloc[coord_list[i]].split(',')
            filename = f'{state}_{city}_{coord_name[i]}'
            print(filename)

            lon = float(coord[0])
            lat = float(coord[1])
            process_point_svm(lon, lat, city, state, '2019', filename)
            process_point_svm(lon, lat, city, state, '2024', filename)
        except Exception as e:
            print(f"Error processing {state}, {city}, {coord_name[i]}: {e}")
            print(f"Exception details: {type(e)._name_} - {e}")
    print()

print("All tasks have been started.")

### Run (without Error handling)

In [7]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]
# Process each city
for j in range(ncities):
  cityData = data.iloc[j]
  city = cityData[1]
  state = cityData[0]
  # print(cityData)
  print(city, state)
  for i in range(9):
      coord = cityData.iloc[coord_list[i]].split(',')
      filename = f'{state}_{city}_{coord_name[i]}'
      print(filename, end=';')

      lon = float(coord[0])
      lat = float(coord[1])
      print(lon, lat)
      # process_point(lon, lat, city, state, '2019', filename)
      # process_point(lon, lat, city, state, '2024', filename)
      process_point_svm(lon, lat, city, state, '2019', filename)
      process_point_svm(lon, lat, city, state, '2024', filename)
  print()

print("All tasks have been started.")